In [1]:
from google.colab import files
uploaded = files.upload()

Saving Final_Dataset_V2.xlsx to Final_Dataset_V2.xlsx


In [7]:
del df

In [8]:
import pandas as pd
# Load data
df = pd.read_excel('Final_Dataset_V2.xlsx', sheet_name='Sheet1')

In [12]:
import pandas as pd
import re
import html
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define the target variable
y = df['rating']

# Define the feature variable
X = df['review']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def preprocess_text(text):
    # Convert to string if input is not a string
    if not isinstance(text, str):
        text = str(text)

    # Remove Original Keywords
    text = text.split("(Original)")[0].rstrip("(").rstrip()

    # Remove Translated by Google
    text = re.sub(r'\(Translated by Google\)', '', text)

    # Decode HTML tags
    text = html.unescape(text)

    # Remove Foul words
    text = re.sub(r'\b(fuck|fucked|fucking|shit|shitty)\b', '', text, flags=re.IGNORECASE)

    # Remove HTTPS
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    # Remove URL
    url_pattern = re.compile(r'(https?://|www\.)\S+')
    text = url_pattern.sub('', text)

    # Remove // & # characters
    text = text.replace('//', '').replace('#', '')

    # Replace **** with BAD
    text = re.sub(r'\*+', 'BAD', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join the tokens back into a string
    text = ' '.join(tokens)

    return text

# Apply the preprocessing function to the training data
X_train = X_train.apply(preprocess_text)

# Apply the preprocessing function to the testing data
X_test = X_test.apply(preprocess_text)

# Create the pipeline
lr_tfidf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr', LogisticRegression())
])

# Train the model
lr_tfidf_pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lr_tfidf_pipeline.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Accuracy: 0.5646666666666667


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
accuracy

0.5646666666666667

In [ ]:
# Create a new DataFrame with the original text and predicted labels
predictions_df = pd.DataFrame({'text': X_test, 'predicted_label': y_pred})

# Join with the original DataFrame to get the actual labels
predictions_df = predictions_df.join(y_test, how='outer')
predictions_df.rename(columns={'label': 'actual_label'}, inplace=True)

# View the first few rows
predictions_df.head()